# 메이플스토리 챌린저스 서버 레벨 260+ 유저 데이터 추출 (7월 16일)

## 목표
- 챌린저스, 챌린저스2, 챌린저스3, 챌린저스4 서버에서 레벨 260 이상인 모든 유저의 정보를 추출
- 각 서버별로 CSV 파일로 저장

## 주의사항
- 넥슨 오픈 API 사용 (API 키 필요)
- 실행 시간이 오래 걸릴 수 있음 (완전한 데이터 추출을 위해)
- API 호출 제한을 준수하기 위해 적절한 딜레이 포함

## 1. 라이브러리 임포트 및 설정

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")

## 2. API 설정 및 클래스 정의

In [ ]:
class MapleStoryChallengersExtractor:
    """
    메이플스토리 챌린저스 서버 데이터 추출기 (다중 서버 지원)
    """

    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://open.api.nexon.com"
        self.headers = {
            "x-nxopen-api-key": api_key,
            "Content-Type": "application/json"
        }
        self.min_level = 260
        self.target_date = "2025-07-16"  # 7월 16일로 고정
        
        # 4개 서버 정의
        self.servers = ["챌린저스", "챌린저스2", "챌린저스3", "챌린저스4"]

    def get_ranking_page(self, world_name: str, page: int) -> Optional[Dict]:
        """
        특정 페이지의 랭킹 데이터 조회

        Args:
            world_name (str): 서버명
            page (int): 페이지 번호

        Returns:
            Dict: API 응답 데이터 또는 None
        """
        url = f"{self.base_url}/maplestory/v1/ranking/overall"

        params = {
            "date": self.target_date,
            "world_name": world_name,
            "page": page
        }

        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"❌ {world_name} 페이지 {page} 조회 실패: {e}")
            return None

    def extract_server_users_above_level(self, world_name: str, delay_seconds: float = 0.2) -> List[Dict]:
        """
        특정 서버에서 레벨 260 이상의 모든 유저 데이터 추출

        Args:
            world_name (str): 서버명
            delay_seconds (float): API 호출 간 딜레이 (초)

        Returns:
            List[Dict]: 모든 유저 데이터
        """
        print(f"🚀 {world_name} 서버에서 레벨 {self.min_level}+ 유저 데이터 추출을 시작합니다...")
        print(f"📅 기준일: {self.target_date}")
        print(f"⏱️  API 호출 간격: {delay_seconds}초")

        page = 1
        total_users = 0
        total_api_calls = 0
        all_users_data = []

        while True:
            print(f"📄 {world_name} 페이지 {page} 조회 중... (현재 발견: {total_users}명)")

            # API 호출
            ranking_data = self.get_ranking_page(world_name, page)
            total_api_calls += 1

            if not ranking_data or 'ranking' not in ranking_data:
                print(f"⚠️ {world_name} 페이지 {page}에서 데이터를 가져올 수 없습니다.")
                break

            current_page_users = ranking_data['ranking']

            # 빈 페이지 체크
            if not current_page_users:
                print(f"📋 {world_name} 페이지 {page}가 비어있습니다. 조회를 종료합니다.")
                break

            # 레벨 필터링
            page_filtered_users = []
            below_level_found = False

            for user in current_page_users:
                user_level = user.get('character_level', 0)

                if user_level >= self.min_level:
                    # 서버 정보 추가
                    user['server_name'] = world_name
                    user['extracted_at'] = datetime.now().isoformat()
                    user['page_number'] = page
                    page_filtered_users.append(user)
                else:
                    below_level_found = True
                    print(f"✅ {world_name} 레벨 {self.min_level} 미만 유저 발견 (레벨 {user_level}). 추출 완료!")
                    break

            # 현재 페이지에서 찾은 유저들 추가
            all_users_data.extend(page_filtered_users)
            total_users += len(page_filtered_users)

            print(f"   ➤ {world_name} 이번 페이지에서 {len(page_filtered_users)}명 발견")

            # 레벨 미만 유저가 발견되면 종료
            if below_level_found:
                break

            # 다음 페이지로
            page += 1

            # API 호출 제한 준수를 위한 딜레이
            time.sleep(delay_seconds)

        print(f"\n🎉 {world_name} 서버 추출 완료!")
        print(f"📊 총 발견 유저: {total_users}명")
        print(f"📞 총 API 호출: {total_api_calls}회")
        print(f"⏱️  소요 시간: 약 {total_api_calls * delay_seconds:.1f}초\n")

        return all_users_data

    def extract_all_servers(self, delay_seconds: float = 0.2) -> Dict[str, List[Dict]]:
        """
        모든 챌린저스 서버에서 데이터 추출
        
        Returns:
            Dict[str, List[Dict]]: 서버별 유저 데이터
        """
        all_server_data = {}
        
        for server in self.servers:
            print(f"\n{'='*50}")
            print(f"시작: {server} 서버 데이터 추출")
            print(f"{'='*50}")
            
            server_data = self.extract_server_users_above_level(server, delay_seconds)
            all_server_data[server] = server_data
            
            print(f"완료: {server} 서버 - {len(server_data)}명 추출")
            
            # 서버 간 딜레이 (과부하 방지)
            if server != self.servers[-1]:  # 마지막 서버가 아니면
                print(f"다음 서버까지 대기 중... (5초)")
                time.sleep(5)
        
        return all_server_data

    def save_to_csv(self, server_data: Dict[str, List[Dict]]) -> Dict[str, str]:
        """
        서버별 데이터를 CSV 파일로 저장
        
        Args:
            server_data (Dict): 서버별 유저 데이터
            
        Returns:
            Dict[str, str]: 서버별 저장된 파일명
        """
        saved_files = {}
        
        for server, users in server_data.items():
            if not users:
                print(f"⚠️ {server}: 저장할 데이터가 없습니다.")
                continue
                
            # DataFrame 생성
            df = pd.DataFrame(users)
            
            # 파일명 생성 (서버별로 구분)
            filename = f"challengers_level_above260_0716_{server}.csv"
            
            try:
                # CSV 저장 (한글 지원)
                df.to_csv(filename, index=False, encoding='utf-8-sig')
                saved_files[server] = filename
                print(f"✅ {server}: '{filename}' 파일로 저장 완료 ({len(users)}명)")
                
                # 기본 통계 출력
                if 'character_level' in df.columns:
                    print(f"   - 최고 레벨: {df['character_level'].max()}")
                    print(f"   - 최저 레벨: {df['character_level'].min()}")
                    print(f"   - 평균 레벨: {df['character_level'].mean():.1f}")
                
            except Exception as e:
                print(f"❌ {server} CSV 저장 실패: {e}")
        
        return saved_files

# API 키 설정
API_KEY = "live_24e7ede45f168bf4b039b07677f76aa58894c93ab408f4bd96ca1e60aaf6932cefe8d04e6d233bd35cf2fabdeb93fb0d"

# 추출기 인스턴스 생성
extractor = MapleStoryChallengersExtractor(API_KEY)

print("✅ 메이플스토리 챌린저스 추출기 준비 완료")
print(f"🌍 대상 서버: {extractor.servers}")
print(f"📈 최소 레벨: {extractor.min_level}")
print(f"📅 기준일: {extractor.target_date}")

## 3. 모든 서버 데이터 추출 실행

⚠️ **주의**: 이 셀은 실행 시간이 매우 오래 걸릴 수 있습니다. 4개 서버의 모든 260+ 레벨 유저를 완전히 추출합니다.

In [ ]:
# 데이터 추출 시작 (서버간 딜레이 포함)
print("🎯 4개 챌린저스 서버 데이터 추출을 시작합니다...")
print(f"📅 추출 대상일: {extractor.target_date}")
print(f"🔢 추출 대상 레벨: {extractor.min_level}+")
print("⏱️ 예상 소요 시간: 약 20-30분 (서버당 5-8분)\n")

start_time = datetime.now()

# 모든 서버 데이터 추출
all_server_data = extractor.extract_all_servers(delay_seconds=0.15)

end_time = datetime.now()
elapsed_time = end_time - start_time

print("\n" + "="*60)
print("🎉 전체 서버 데이터 추출 완료!")
print("="*60)
print(f"⏱️ 총 소요 시간: {elapsed_time}")

# 서버별 요약 통계
total_users = 0
for server, users in all_server_data.items():
    user_count = len(users)
    total_users += user_count
    print(f"📊 {server}: {user_count:,}명")

print(f"\n🔢 전체 추출 유저 수: {total_users:,}명")

## 4. CSV 파일 저장

In [ ]:
# CSV 파일로 저장
print("💾 CSV 파일 저장을 시작합니다...\n")

saved_files = extractor.save_to_csv(all_server_data)

print("\n" + "="*50)
print("📁 저장 완료된 파일 목록:")
print("="*50)

for server, filename in saved_files.items():
    print(f"✅ {server}: {filename}")

print(f"\n🎯 총 {len(saved_files)}개 CSV 파일이 생성되었습니다.")

## 5. 통합 데이터 분석

In [ ]:
# 모든 서버 데이터를 하나로 통합
all_users = []
for server, users in all_server_data.items():
    all_users.extend(users)

if all_users:
    # 통합 DataFrame 생성
    df_all = pd.DataFrame(all_users)
    
    print("📊 전체 서버 통합 분석 결과:")
    print("="*40)
    print(f"총 데이터 수: {len(df_all):,}개")
    print(f"컬럼 수: {len(df_all.columns)}개")
    
    # 서버별 분포
    print("\n🌍 서버별 유저 분포:")
    server_counts = df_all['server_name'].value_counts()
    for server, count in server_counts.items():
        print(f"  {server}: {count:,}명")
    
    # 레벨 통계
    print("\n📈 레벨 통계 (전체):")
    level_stats = df_all['character_level'].describe()
    print(f"  평균 레벨: {level_stats['mean']:.1f}")
    print(f"  최고 레벨: {int(level_stats['max'])}")
    print(f"  최저 레벨: {int(level_stats['min'])}")
    print(f"  중간값: {level_stats['50%']:.1f}")
    
    # 직업별 분포 (상위 10개)
    print("\n⚔️ 직업별 분포 (상위 10개):")
    job_counts = df_all['class_name'].value_counts().head(10)
    for job, count in job_counts.items():
        percentage = (count / len(df_all)) * 100
        print(f"  {job}: {count:,}명 ({percentage:.1f}%)")
    
    # 통합 CSV 저장
    combined_filename = "challengers_level_above260_0716_ALL_SERVERS.csv"
    df_all.to_csv(combined_filename, index=False, encoding='utf-8-sig')
    print(f"\n💾 통합 파일 저장: '{combined_filename}'")
    
else:
    print("❌ 분석할 데이터가 없습니다.")

## 6. 최종 요약

In [ ]:
print("🎉 데이터 추출 및 저장 작업이 모두 완료되었습니다!")
print("="*60)

print(f"📅 추출 기준일: {extractor.target_date}")
print(f"🎯 추출 조건: 레벨 {extractor.min_level} 이상")
print(f"🌍 대상 서버: {', '.join(extractor.servers)}")

if saved_files:
    print("\n📁 생성된 파일:")
    for server, filename in saved_files.items():
        user_count = len(all_server_data[server])
        print(f"  ✅ {filename} ({user_count:,}명)")
    
    if 'combined_filename' in locals():
        print(f"  ✅ {combined_filename} (전체 {total_users:,}명)")

print("\n⚠️ 주의사항:")
print("  - CSV 파일은 UTF-8 BOM 인코딩으로 저장되어 Excel에서 한글이 정상 표시됩니다")
print("  - 데이터는 7월 16일 기준으로 추출되었습니다")
print("  - API 호출 제한을 준수하여 안전하게 추출되었습니다")

print("\n🔚 작업 완료!")